<a href="https://colab.research.google.com/github/atharvajoshi10/FederatedML/blob/main/Roberta_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base Model

Attempting to recreate the validation F1 of the following model using federated learning.
https://huggingface.co/JeremiahZ/roberta-base-mrpc

# Dependencies

GPU - T4

Note : Force Update accelerate since it defaults to the old version.
Restart the notebook once the below cell has executed.

In [1]:
%%capture
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install accelerate -U

In [2]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, AutoConfig, MobileBertConfig, TrainingArguments, DataCollatorWithPadding
import numpy as np
import evaluate

# Defining Training Functions
- Import dataset
- Import Base Model and tokenizer
- Define Training Hyperparameters
- Train Model

In [3]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def train_model(train_path, train_file_name, validation_path, validation_file_name, model_save_path, dataset_id):

  train_dataset = load_from_disk(train_path+dataset_id+train_file_name)
  validation_dataset = load_from_disk(validation_path+dataset_id+validation_file_name)

  tokenizer = AutoTokenizer.from_pretrained("roberta-base")
  model = AutoModelForSequenceClassification.from_pretrained("roberta-base")

  def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"])

  tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
  tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  training_args = TrainingArguments("estimate_trainer/"+dataset_id, num_train_epochs=5, learning_rate=2e-05,
                                  lr_scheduler_type="linear", per_device_train_batch_size=16, seed=42,
                                  per_device_eval_batch_size=8, warmup_ratio=0.06)

  trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
    )
  trainer.train()

  predictions = trainer.predict(tokenized_validation_dataset)
  preds = np.argmax(predictions.predictions, axis=-1)
  metric = evaluate.load("glue", "mrpc")
  validation_scores.append(metric.compute(predictions=preds, references=predictions.label_ids))
  trainer.save_model(model_save_path+"estimate_trainer/"+dataset_id)


Passing all the paths and calling on the sharded dataset.

In [4]:
train_path = "/content/drive/MyDrive/FML Data/Federated Split/train/"
validation_path = "/content/drive/MyDrive/FML Data/Federated Split/validation/"
model_save_path = "/content/drive/MyDrive/FML Data/Federated Models/"

train_file_name = "/train.hf"
validation_file_name = "/validation.hf"

validation_scores = []

num_shards = 8

for dataset_id in range(num_shards):
  train_model(train_path, train_file_name, validation_path, validation_file_name, model_save_path, str(dataset_id))
  print(validation_scores[-1])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


{'accuracy': 0.7647058823529411, 'f1': 0.8571428571428571}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


...

Step,Training Loss


{'accuracy': 0.7843137254901961, 'f1': 0.8405797101449276}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


{'accuracy': 0.8627450980392157, 'f1': 0.9014084507042254}
